# <font color="#418FDE" size="6.5" uppercase>**CNN Fundamentals**</font>

>Last update: 20260126.
    
By the end of this Lecture, you will be able to:
- Construct basic CNN architectures using Conv2D, MaxPooling2D, and Dense layers in tf.keras. 
- Prepare image datasets with tf.data and image preprocessing utilities for CNN training. 
- Train and evaluate a CNN on a small image dataset, interpreting accuracy and loss metrics. 


## **1. Convolution Layer Basics**

### **1.1. Understanding Filters and Strides**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_01_01.jpg?v=1769409644" width="250">



>* Filters slide over images detecting local patterns
>* Many filters create deep feature maps for representation

>* Stride controls filter movement and feature resolution
>* Larger strides reduce detail but save computation

>* Filters and strides balance detail and efficiency
>* More filters, small strides capture rich, useful patterns



In [ ]:
#@title Python Code - Understanding Filters and Strides

# This script explains filters and strides visually.
# It uses TensorFlow to build simple Conv2D layers.
# We compare outputs for different filters and strides.

# !pip install tensorflow==2.20.0.

# Import required libraries safely.
import os
import random
import numpy as np
import tensorflow as tf

# Set deterministic random seeds.
seed_value = 7
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Create a tiny 6x6 grayscale image.
base_image = np.array(
    [
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ],
    dtype="float32",
)

# Expand dimensions to match Conv2D input.
image_tensor = base_image[np.newaxis, ..., np.newaxis]

# Validate the image tensor shape.
print("Input image shape:", image_tensor.shape)

# Define a vertical edge detection filter.
vertical_filter = np.array(
    [
        [-1.0, 1.0, 0.0],
        [-1.0, 1.0, 0.0],
        [-1.0, 1.0, 0.0],
    ],
    dtype="float32",
)

# Define a horizontal edge detection filter.
horizontal_filter = np.array(
    [
        [-1.0, -1.0, -1.0],
        [1.0, 1.0, 1.0],
        [0.0, 0.0, 0.0],
    ],
    dtype="float32",
)

# Stack filters to create two filter kernels.
kernel_stack = np.stack([vertical_filter, horizontal_filter], axis=-1)

# Reshape kernel to match Conv2D kernel shape.
kernel_tensor = kernel_stack[..., np.newaxis, :]

# Confirm kernel tensor shape for safety.
print("Kernel tensor shape:", kernel_tensor.shape)

# Build a Conv2D layer with stride one.
conv_stride_one = tf.keras.layers.Conv2D(
    filters=2,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="valid",
    use_bias=False,
)

# Build a Conv2D layer with stride two.
conv_stride_two = tf.keras.layers.Conv2D(
    filters=2,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="valid",
    use_bias=False,
)

# Call layers once to build weights.
_ = conv_stride_one(image_tensor)
_ = conv_stride_two(image_tensor)

# Assign our custom kernel to both layers.
conv_stride_one.set_weights([kernel_tensor])
conv_stride_two.set_weights([kernel_tensor])

# Apply convolution with stride one.
output_one = conv_stride_one(image_tensor).numpy()

# Apply convolution with stride two.
output_two = conv_stride_two(image_tensor).numpy()

# Print shapes to show stride effect.
print("Output shape stride 1:", output_one.shape)
print("Output shape stride 2:", output_two.shape)

# Print small feature map for first filter.
print("Feature map filter 1 stride 1:")
print(output_one[0, :, :, 0])

# Print small feature map for first filter stride two.
print("Feature map filter 1 stride 2:")
print(output_two[0, :, :, 0])

# Final confirmation line summarizing lesson.
print("More filters learn patterns, larger strides reduce resolution.")



### **1.2. Padding and Kernel Choices**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_01_02.jpg?v=1769409721" width="250">



>* Kernel size controls local versus global patterns
>* Padding preserves edge information and output size

>* Valid padding shrinks feature maps, emphasizing compression
>* Same padding keeps size, preserving spatial detail

>* Small kernels with same padding grow receptive field
>* Kernel and padding choices trade detail for context



In [ ]:
#@title Python Code - Padding and Kernel Choices

# This script shows padding and kernel basics.
# It compares Conv2D outputs with different settings.
# It uses a tiny random image batch example.

# !pip install tensorflow==2.20.0.

# Import required modules from TensorFlow.
import tensorflow as tf

# Set a deterministic random seed value.
tf.random.set_seed(7)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Create a tiny batch of random grayscale images.
images = tf.random.uniform((2, 8, 8, 1))

# Confirm the input tensor shape is correct.
print("Input shape:", images.shape)

# Build a Conv2D layer with same padding.
conv_same = tf.keras.layers.Conv2D(
    filters=1,
    kernel_size=(3, 3),
    padding="same",
)

# Build a Conv2D layer with valid padding.
conv_valid = tf.keras.layers.Conv2D(
    filters=1,
    kernel_size=(3, 3),
    padding="valid",
)

# Apply same padding convolution to the images.
output_same = conv_same(images)

# Apply valid padding convolution to the images.
output_valid = conv_valid(images)

# Show output shapes for both padding choices.
print("Same padding output shape:", output_same.shape)

# Show how valid padding reduces spatial dimensions.
print("Valid padding output shape:", output_valid.shape)

# Build a Conv2D layer with larger kernel size.
conv_large_kernel = tf.keras.layers.Conv2D(
    filters=1,
    kernel_size=(5, 5),
    padding="same",
)

# Apply the larger kernel convolution to images.
output_large_kernel = conv_large_kernel(images)

# Confirm shape stays same with same padding.
print("Large kernel same padding shape:", output_large_kernel.shape)

# Compare parameter counts for small and large kernels.
params_small = conv_same.count_params()

# Compute parameter count for the larger kernel layer.
params_large = conv_large_kernel.count_params()

# Print parameter counts to show kernel size effect.
print("Params small kernel:", params_small)

# Print parameter counts for the larger kernel.
print("Params large kernel:", params_large)

# End by printing a short explanatory summary line.
print("Same keeps size, valid shrinks; larger kernels add parameters.")



### **1.3. CNN Activation Functions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_01_03.jpg?v=1769409753" width="250">



>* Activation functions add nonlinearity so CNNs learn
>* They gate feature strength, building complex visual concepts

>* ReLU is default, fast, and avoids vanishing gradients
>* It creates sparse, focused activations highlighting key patterns

>* ReLU has alternatives like leaky and parametric
>* Activation choice and placement strongly affect feature separation



In [ ]:
#@title Python Code - CNN Activation Functions

# This script demonstrates CNN activation functions.
# It uses TensorFlow to build tiny models.
# Focus on ReLU and sigmoid layer behaviors.

# !pip install tensorflow==2.20.0.

# Import required libraries safely.
import os
import random
import numpy as np
import tensorflow as tf

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Create a small batch of fake image data.
batch_size = 4
height, width, channels = 8, 8, 1
images = np.random.randn(batch_size, height, width, channels).astype("float32")

# Confirm the image batch shape is correct.
print("Image batch shape:", images.shape)

# Build a simple CNN block with ReLU.
relu_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=2,
        kernel_size=(3, 3),
        activation="relu",
        input_shape=(height, width, channels),
    )
])

# Build a similar CNN block with sigmoid.
sigmoid_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=2,
        kernel_size=(3, 3),
        activation="sigmoid",
        input_shape=(height, width, channels),
    )
])

# Run a forward pass through the ReLU model.
relu_output = relu_model(images)

# Run a forward pass through the sigmoid model.
sigmoid_output = sigmoid_model(images)

# Check that output shapes match expectations.
print("ReLU output shape:", relu_output.shape)
print("Sigmoid output shape:", sigmoid_output.shape)

# Compute simple statistics for ReLU activations.
relu_min = float(tf.reduce_min(relu_output).numpy())
relu_max = float(tf.reduce_max(relu_output).numpy())
relu_mean = float(tf.reduce_mean(relu_output).numpy())

# Compute simple statistics for sigmoid activations.
sig_min = float(tf.reduce_min(sigmoid_output).numpy())
sig_max = float(tf.reduce_max(sigmoid_output).numpy())
sig_mean = float(tf.reduce_mean(sigmoid_output).numpy())

# Print summary statistics for both activations.
print("ReLU activations min max mean:", relu_min, relu_max, relu_mean)
print("Sigmoid activations min max mean:", sig_min, sig_max, sig_mean)

# Show how many ReLU outputs are exactly zero.
zero_count = int(tf.math.count_nonzero(relu_output == 0.0).numpy())
print("Number of exact zeros in ReLU output:", zero_count)




## **2. Pooling and Flattening**

### **2.1. Pooling Layer Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_02_01.jpg?v=1769409789" width="250">



>* Pooling summarizes and compresses spatial feature information
>* Max pooling keeps strongest activations, reducing resolution

>* Pooling makes features less sensitive to position
>* It improves robustness to shifts, distortions, and noise

>* Pooling shrinks feature maps, saving compute and memory
>* Balances speed and detail, reducing overfitting risk



In [ ]:
#@title Python Code - Pooling Layer Basics

# This script demonstrates basic pooling concepts.
# It uses TensorFlow to build simple layers.
# Focus is on pooling and flattening behavior.

# !pip install tensorflow==2.20.0.

# Import TensorFlow and NumPy for this demo.
import tensorflow as tf
import numpy as np

# Print TensorFlow version for reproducibility.
print("TensorFlow version:", tf.__version__)

# Set random seeds for deterministic behavior.
np.random.seed(42)
tf.random.set_seed(42)

# Create a small fake image batch tensor.
input_batch = tf.constant(
    np.arange(1, 17, dtype=np.float32).reshape((1, 4, 4, 1))
)

# Confirm the input shape is as expected.
print("Input batch shape:", input_batch.shape)

# Print the input values in a clear layout.
print("Input values (4x4 image):")
print(tf.reshape(input_batch, (4, 4)))

# Define a max pooling layer with 2x2 window.
max_pool = tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=(2, 2)
)

# Apply max pooling to the input batch.
pooled_output = max_pool(input_batch)

# Show pooled output shape and values.
print("Pooled shape:", pooled_output.shape)
print("Pooled values (2x2 result):")
print(tf.reshape(pooled_output, (2, 2)))

# Define an average pooling layer for comparison.
avg_pool = tf.keras.layers.AveragePooling2D(
    pool_size=(2, 2), strides=(2, 2)
)

# Apply average pooling to the same input.
avg_output = avg_pool(input_batch)

# Show average pooled shape and values.
print("Average pooled shape:", avg_output.shape)
print("Average pooled values (2x2):")
print(tf.reshape(avg_output, (2, 2)))

# Define a flatten layer to go to Dense.
flatten_layer = tf.keras.layers.Flatten()

# Flatten the max pooled output tensor.
flattened = flatten_layer(pooled_output)

# Show flattened shape and values for clarity.
print("Flattened shape:", flattened.shape)
print("Flattened values:", flattened.numpy())




### **2.2. Flatten vs GlobalAveragePooling2D**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_02_02.jpg?v=1769409820" width="250">



>* Flattening keeps every spatial activation for classification
>* Global average pooling summarizes each feature map overall

>* Flattening creates huge vectors, many parameters, overfitting
>* Global average pooling reduces parameters, improves generalization

>* Flatten keeps spatial detail but less robust
>* Global pooling adds invariance and guides preprocessing choices



In [ ]:
#@title Python Code - Flatten vs GlobalAveragePooling2D

# This script compares Flatten and GlobalAveragePooling2D.
# It uses a tiny CNN on MNIST images.
# Focus is on dataset prep and pooling choices.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Set deterministic seeds for reproducibility.
random.seed(42)
np.random.seed(42)

# Import TensorFlow and Keras layers.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Check and report available physical devices.
devices = tf.config.list_physical_devices()
print("Available devices count:", len(devices))

# Load MNIST dataset from Keras datasets.
(mnist_x_train, mnist_y_train), _ = keras.datasets.mnist.load_data()

# Select a small subset for quick demonstration.
subset_size = 4000
mnist_x_train = mnist_x_train[:subset_size]
mnist_y_train = mnist_y_train[:subset_size]

# Normalize pixel values to range [0,1].
mnist_x_train = mnist_x_train.astype("float32") / 255.0

# Add channel dimension for CNN input.
mnist_x_train = np.expand_dims(mnist_x_train, axis=-1)

# Confirm input shape is as expected.
print("Train subset shape:", mnist_x_train.shape)

# Create a tf.data Dataset from numpy arrays.
train_ds = tf.data.Dataset.from_tensor_slices(
    (mnist_x_train, mnist_y_train)
)

# Shuffle dataset with a safe buffer size.
train_ds = train_ds.shuffle(buffer_size=subset_size, seed=42)

# Define a simple preprocessing function.
def preprocess(image, label):
    image = tf.image.resize(image, size=(28, 28))
    return image, label

# Map preprocessing over the dataset.
train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Batch and prefetch for efficient input pipeline.
batch_size = 64
train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build a small shared convolutional base.
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(16, (3, 3), activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Add another convolution and pooling layer.
x = layers.Conv2D(32, (3, 3), activation="relu")(x)
feature_maps = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Create Flatten branch for classification.
flat_branch = layers.Flatten()(feature_maps)
flat_output = layers.Dense(10, activation="softmax")(flat_branch)

# Create GlobalAveragePooling2D branch for classification.
gap_branch = layers.GlobalAveragePooling2D()(feature_maps)
gap_output = layers.Dense(10, activation="softmax")(gap_branch)

# Build two separate models sharing the same base.
model_flat = keras.Model(inputs=inputs, outputs=flat_output)
model_gap = keras.Model(inputs=inputs, outputs=gap_output)

# Compile both models with same optimizer and loss.
model_flat.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Compile the global average pooling model similarly.
model_gap.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Train Flatten model briefly with silent verbose.
history_flat = model_flat.fit(train_ds, epochs=1, verbose=0)

# Train GlobalAveragePooling2D model briefly.
history_gap = model_gap.fit(train_ds, epochs=1, verbose=0)

# Get parameter counts for both models.
params_flat = model_flat.count_params()
params_gap = model_gap.count_params()

# Extract final training accuracy values.
acc_flat = history_flat.history["accuracy"][-1]
acc_gap = history_gap.history["accuracy"][-1]

# Print concise comparison of parameter counts.
print("Flatten model parameters:", params_flat)
print("GAP model parameters:", params_gap)

# Print concise comparison of training accuracies.
print("Flatten model train accuracy:", round(float(acc_flat), 4))
print("GAP model train accuracy:", round(float(acc_gap), 4))

# Show feature map and pooled shapes for intuition.
print("Feature maps shape example:", feature_maps.shape)
print("Flatten output length:", flat_branch.shape[-1])
print("GAP output length:", gap_branch.shape[-1])




### **2.3. Managing Feature Map Size**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_02_03.jpg?v=1769409863" width="250">



>* Feature map size affects memory and computation
>* Balance downsampling to keep detail yet efficiency

>* Control feature maps using input size and layers
>* Balance downsampling speed with preserving important details

>* Standardize image sizes and batches during preprocessing
>* Plan layer downsizing to balance efficiency and detail



In [ ]:
#@title Python Code - Managing Feature Map Size

# This script shows managing CNN feature map size.
# We use MNIST images with simple preprocessing steps.
# We inspect shapes after pooling and flattening.

# !pip install tensorflow==2.20.0.

# Import required libraries safely.
import os
import random
import numpy as np
import tensorflow as tf

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Load MNIST dataset from tf.keras datasets.
(mnist_x_train, mnist_y_train), _ = tf.keras.datasets.mnist.load_data()

# Select a small subset for quick processing.
subset_size = 512
mnist_x_train = mnist_x_train[:subset_size]
mnist_y_train = mnist_y_train[:subset_size]

# Add channel dimension and convert to float.
mnist_x_train = mnist_x_train[..., np.newaxis].astype("float32")

# Normalize pixel values to range zero one.
mnist_x_train = mnist_x_train / 255.0

# Define target image size for the pipeline.
img_height, img_width = 28, 28

# Create a tf.data dataset from numpy arrays.
train_ds = tf.data.Dataset.from_tensor_slices(
    (mnist_x_train, mnist_y_train)
)

# Define a simple preprocessing function.
def preprocess(image, label):
    # Ensure image has expected shape.
    image = tf.ensure_shape(image, (28, 28, 1))
    # Resize image to target spatial size.
    image = tf.image.resize(image, (img_height, img_width))
    return image, label

# Apply preprocessing and batching to dataset.
train_ds = train_ds.map(preprocess).batch(32)

# Take one small batch for shape inspection.
for batch_images, batch_labels in train_ds.take(1):
    input_batch = batch_images

# Print original batch and image shapes.
print("Input batch shape:", input_batch.shape)
print("Single image shape:", input_batch[0].shape)

# Build a small CNN to track feature sizes.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(img_height, img_width, 1)),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax"),
])

# Call the model once to define its input tensor.
_ = model(input_batch)

# Run a forward pass to observe feature shapes.
feature_extractor = tf.keras.Model(
    inputs=model.layers[0].input,
    outputs=[
        model.layers[1].output,
        model.layers[2].output,
        model.layers[3].output,
        model.layers[4].output,
        model.layers[5].output,
    ],
)

# Compute intermediate feature maps for one batch.
conv1_out, pool1_out, conv2_out, pool2_out, flat_out = feature_extractor(
    input_batch
)

# Print shapes after each important transformation.
print("After first conv shape:", conv1_out.shape)
print("After first pool shape:", pool1_out.shape)
print("After second conv shape:", conv2_out.shape)
print("After second pool shape:", pool2_out.shape)
print("After flatten shape:", flat_out.shape)

# Show how many features remain after flattening.
print("Flattened feature vector length:", flat_out.shape[-1])



## **3. Building Image Pipelines**

### **3.1. Loading Images From Directories**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_03_01.jpg?v=1769409968" width="250">



>* Load images from labeled folders into tensors
>* Ensure consistent shapes and easy dataset swapping

>* Resize images and assign consistent numeric labels
>* Carefully split data to avoid validation leakage

>* Use batched, shuffled pipelines for training images
>* Disable shuffling for stable, trustworthy evaluation metrics



In [ ]:
#@title Python Code - Loading Images From Directories

# This script shows loading images from directories.
# It uses TensorFlow image utilities for beginners.
# It keeps output short and training very small.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import pathlib
import random

# Import TensorFlow and Keras utilities.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)

# Set TensorFlow random seed for determinism.
tf.random.set_seed(seed_value)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Detect available device type for information.
physical_gpus = tf.config.list_physical_devices("GPU")

# Print whether GPU is available or only CPU.
print("GPU available:", bool(physical_gpus))

# Load CIFAR10 dataset from Keras datasets.
(cifar_images, cifar_labels), _ = keras.datasets.cifar10.load_data()

# Confirm dataset shapes before further processing.
print("CIFAR10 images shape:", cifar_images.shape)

# Choose a tiny subset for quick demonstration.
subset_size = 60
cifar_images = cifar_images[:subset_size]

# Slice labels to match the image subset.
cifar_labels = cifar_labels[:subset_size].flatten()

# Validate subset shapes after slicing.
print("Subset images shape:", cifar_images.shape)

# Create a base directory for image folders.
base_dir = pathlib.Path("cifar10_small_dir")

# Remove existing directory tree if it exists.
if base_dir.exists():
    for child in base_dir.rglob("*"):
        if child.is_file():
            child.unlink()

# Recreate the base directory cleanly.
if base_dir.exists():
    for child in sorted(base_dir.glob("*")):
        if child.is_dir():
            for sub in child.glob("*"):
                if sub.is_file():
                    sub.unlink()

# Ensure base directory exists for saving.
base_dir.mkdir(exist_ok=True)

# Define class names for first three classes.
class_names = ["airplane", "automobile", "bird"]

# Create subdirectories for train and validation.
train_dir = base_dir / "train"
val_dir = base_dir / "val"

# Make train and validation directories.
train_dir.mkdir(exist_ok=True)
val_dir.mkdir(exist_ok=True)

# Create class subfolders inside train directory.
for name in class_names:
    (train_dir / name).mkdir(exist_ok=True)

# Create class subfolders inside validation directory.
for name in class_names:
    (val_dir / name).mkdir(exist_ok=True)

# Separate indices for selected three classes.
selected_indices = [i for i, y in enumerate(cifar_labels)
                    if int(y) in [0, 1, 2]]

# Limit to small number of images per class.
max_per_class = 10

# Track counts per class for balancing.
class_counts = {0: 0, 1: 0, 2: 0}

# Iterate indices and save images into folders.
for idx in selected_indices:
    label = int(cifar_labels[idx])
    if class_counts[label] >= max_per_class:
        continue
    class_counts[label] += 1
    split = "train" if class_counts[label] <= 7 else "val"
    folder = train_dir if split == "train" else val_dir
    class_name = class_names[label]
    img_array = cifar_images[idx]
    img_path = folder / class_name / f"img_{idx}.png"
    tf.keras.utils.save_img(str(img_path), img_array)

# Print how many images saved per class.
print("Saved per class:", class_counts)

# Define common image size for loading.
img_height = 32
img_width = 32

# Create training dataset from directory.
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_dir,
    labels="inferred",
    label_mode="int",
    image_size=(img_height, img_width),
    batch_size=8,
    shuffle=True,
    seed=seed_value,
    validation_split=None,
)

# Create validation dataset from directory.
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=val_dir,
    labels="inferred",
    label_mode="int",
    image_size=(img_height, img_width),
    batch_size=8,
    shuffle=False,
    seed=seed_value,
    validation_split=None,
)

# Cache and prefetch datasets for performance.
autotune = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=autotune)

# Apply same optimization to validation dataset.
val_ds = val_ds.cache().prefetch(buffer_size=autotune)

# Build a tiny CNN model for demonstration.
model = keras.Sequential([
    layers.Rescaling(1.0 / 255.0, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(32, activation="relu"),
    layers.Dense(len(class_names), activation="softmax"),
])

# Compile model with simple optimizer and loss.
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Train model briefly with silent verbose setting.
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    verbose=0,
)

# Evaluate model on validation dataset silently.
val_loss, val_acc = model.evaluate(val_ds, verbose=0)

# Print final validation loss and accuracy.
print("Validation loss:", round(float(val_loss), 4))

# Print validation accuracy rounded for readability.
print("Validation accuracy:", round(float(val_acc), 4))



### **3.2. Image Rescaling and Augmentation**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_03_02.jpg?v=1769410086" width="250">



>* Rescale pixel values to a consistent range
>* Rescaling stabilizes training and highlights meaningful patterns

>* Augmentation creates varied, realistic versions of images
>* Model learns robust features, improving generalization performance

>* Match rescaling and augmentation to task, metrics
>* Tune augmentation strength by watching accuracy, loss



In [ ]:
#@title Python Code - Image Rescaling and Augmentation

# This script shows image rescaling and augmentation.
# It uses TensorFlow to build simple pipelines.
# Focus on visualizing effects not heavy training.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

# Import TensorFlow and check version.
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Detect available device type for information.
physical_gpus = tf.config.list_physical_devices("GPU")
print("GPUs available:", len(physical_gpus))

# Load MNIST dataset for simple image examples.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
print("Original train shape:", x_train.shape)

# Select a small subset for quick processing.
subset_size = 8
x_small = x_train[:subset_size]
y_small = y_train[:subset_size]

# Expand grayscale images to have channel dimension.
x_small = np.expand_dims(x_small, axis=-1)
print("Subset shape with channel:", x_small.shape)

# Normalize pixel values to range zero one.
x_rescaled = x_small.astype("float32") / 255.0
print("Rescaled min max:", x_rescaled.min(), x_rescaled.max())

# Create a tf.data dataset from rescaled images.
ds_base = tf.data.Dataset.from_tensor_slices((x_rescaled, y_small))
ds_base = ds_base.batch(4)

# Define simple augmentation layer using Keras.
augmentation_layer = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)
])

# Apply augmentation only on training batches.
augmented_examples = []
for batch_images, _ in ds_base.take(1):
    augmented_batch = augmentation_layer(batch_images, training=True)
    augmented_examples.append(augmented_batch)

# Convert one augmented batch to numpy array.
augmented_batch_np = augmented_examples[0].numpy()
print("Augmented batch shape:", augmented_batch_np.shape)

# Verify rescaling is preserved after augmentation.
print("Augmented min max:", augmented_batch_np.min(), augmented_batch_np.max())

# Show first original and augmented pixel center values.
center_index = 14
orig_center = x_rescaled[0, center_index, center_index, 0]
aug_center = augmented_batch_np[0, center_index, center_index, 0]
print("Original center pixel:", float(orig_center))
print("Augmented center pixel:", float(aug_center))

# Build a simple CNN model for demonstration.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(8, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
])

# Compile model with basic optimizer and loss.
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Prepare augmented training dataset pipeline.
ds_train = tf.data.Dataset.from_tensor_slices((x_rescaled, y_small))
ds_train = ds_train.shuffle(subset_size, seed=seed_value)
ds_train = ds_train.map(lambda img, label: (augmentation_layer(img,
                                                               training=True),
                                            label))

# Batch and prefetch for efficient training.
ds_train = ds_train.batch(4).prefetch(tf.data.AUTOTUNE)

# Train briefly with silent verbose setting.
history = model.fit(ds_train, epochs=2, verbose=0)

# Print final training loss and accuracy metrics.
final_loss = history.history["loss"][-1]
final_acc = history.history["accuracy"][-1]
print("Final training loss:", round(final_loss, 4))
print("Final training accuracy:", round(final_acc, 4))




### **3.3. Batching and Shuffling**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_06/Lecture_A/image_03_03.jpg?v=1769410125" width="250">



>* Batching splits data; batch size controls updates
>* Batch size trades memory, speed, and generalization

>* Shuffle images each epoch to randomize order
>* Prevents sequence bias and makes batches representative

>* Small, shuffled batches cause natural metric fluctuations
>* Watch long-term trends to detect overfitting issues



In [ ]:
#@title Python Code - Batching and Shuffling

# This script shows batching and shuffling basics.
# It uses a tiny MNIST subset for clarity.
# We train a small CNN with silent logging.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Set deterministic seeds for reproducibility.
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Import TensorFlow and Keras utilities.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Select device based on GPU availability.
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    device_name = "GPU"
else:
    device_name = "CPU"

# Inform which device will likely be used.
print("Using device:", device_name)

# Load MNIST dataset from Keras datasets.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Reduce dataset size for faster demonstration.
train_samples = 4000
test_samples = 1000

# Slice arrays to keep only small subsets.
x_train = x_train[:train_samples]
y_train = y_train[:train_samples]

# Slice test arrays similarly for quick evaluation.
x_test = x_test[:test_samples]
y_test = y_test[:test_samples]

# Normalize pixel values to range [0, 1].
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Add channel dimension for Conv2D layers.
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Confirm shapes are as expected.
print("Train shape:", x_train.shape)
print("Test shape:", x_test.shape)

# Define batch size hyperparameter.
batch_size = 64

# Create base tf.data dataset from tensors.
base_train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# Demonstrate dataset without shuffling first.
no_shuffle_ds = base_train_ds.batch(batch_size)

# Take one batch and inspect label distribution.
no_shuffle_batch = next(iter(no_shuffle_ds))
print("First batch labels no shuffle:", no_shuffle_batch[1][:10].numpy())

# Now create shuffled and batched dataset.
shuffle_buffer = train_samples
shuffled_ds = base_train_ds.shuffle(shuffle_buffer, seed=SEED)

# Batch after shuffling to mix classes.
shuffled_batched_ds = shuffled_ds.batch(batch_size)

# Inspect first shuffled batch label distribution.
shuffled_batch = next(iter(shuffled_batched_ds))
print("First batch labels shuffled:", shuffled_batch[1][:10].numpy())

# Prefetch to overlap preprocessing and training.
train_ds = shuffled_batched_ds.prefetch(tf.data.AUTOTUNE)

# Prepare a small batched test dataset.
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Batch and prefetch test dataset for efficiency.
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build a simple CNN model for classification.
model = keras.Sequential([
    layers.Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax"),
])

# Compile model with optimizer, loss, and metrics.
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Train model briefly with silent verbose setting.
history = model.fit(
    train_ds,
    epochs=3,
    validation_data=test_ds,
    verbose=0,
)

# Extract final training and validation metrics.
final_train_loss = history.history["loss"][-1]
final_train_acc = history.history["accuracy"][-1]

# Extract final validation loss and accuracy.
final_val_loss = history.history["val_loss"][-1]
final_val_acc = history.history["val_accuracy"][-1]

# Print concise summary of training results.
print("Final train loss:", round(float(final_train_loss), 4))
print("Final train accuracy:", round(float(final_train_acc), 4))
print("Final val loss:", round(float(final_val_loss), 4))
print("Final val accuracy:", round(float(final_val_acc), 4))




# <font color="#418FDE" size="6.5" uppercase>**CNN Fundamentals**</font>


In this lecture, you learned to:
- Construct basic CNN architectures using Conv2D, MaxPooling2D, and Dense layers in tf.keras. 
- Prepare image datasets with tf.data and image preprocessing utilities for CNN training. 
- Train and evaluate a CNN on a small image dataset, interpreting accuracy and loss metrics. 

In the next Lecture (Lecture B), we will go over 'Transfer Learning'